#### Libs

In [1]:
import pandas as pd

DATADIR = '../datasets'

#### Dataset de localidade da rede estadual

In [2]:
df_localiz = pd.read_excel('../datasets/saude_localidade.xlsx', usecols=['codmunres', 'cod_geres', 'geresnome'], dtype={'cod_geres': str})
df_localiz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   codmunres  187 non-null    int64 
 1   cod_geres  185 non-null    object
 2   geresnome  185 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.5+ KB


#### 1. Dataset de Partos por Município de residência por Geres estabelecimento

In [3]:
def transform_df(df):
    df.drop('Total', axis=1, inplace=True)
    df['cd_munic'] = df['Munic Residencia PE'].apply(lambda x: x[:6]).astype(int)
    df['municipio'] = df['Munic Residencia PE'].apply(lambda x: x[7:])
    df = df.merge(df_localiz, how='left', left_on='cd_munic', right_on='codmunres')
    return df

In [4]:
df_partos_risco_munic = pd.read_csv(f"{DATADIR}/partos_gravidez_risco_munic_resid_geres_ocorr.csv", sep=";")
df_partos_risco_munic = transform_df(df_partos_risco_munic)
df_partos_risco_munic['TIPO_PARTO'] = 'De Risco'

df_partos_normal_munic = pd.read_csv(f"{DATADIR}/partos_normais_munic_resid_geres_ocorr.csv", sep=";")
df_partos_normal_munic = transform_df(df_partos_normal_munic)
df_partos_normal_munic['TIPO_PARTO'] = 'Normal'

df_partos_munic = pd.concat([df_partos_risco_munic, df_partos_normal_munic])
df_partos_munic.head()

,Munic Residencia PE,2601 Recife,2604 Caruaru,2608 Petrolina,cd_munic,municipio,codmunres,cod_geres,geresnome,TIPO_PARTO,2602 Limoeiro,2603 Palmares,2605 Garanhuns,2606 Arcoverde,2607 Salgueiro,2609 Ouricuri,2610 Afogados da Ingazeira,2611 Serra Talhada,2612 Goiana
0,260005 Abreu e Lima,540,0,0,260005,Abreu e Lima,260005.0,2601,I GERES - Recife,De Risco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,260010 Afogados da Ingazeira,37,6,0,260010,Afogados da Ingazeira,260010.0,2610,X GERES - Afogados da Ingazeira,De Risco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,260020 Afrânio,0,1,20,260020,Afrânio,260020.0,2608,VIII GERES - Petrolina,De Risco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,260030 Agrestina,63,88,0,260030,Agrestina,260030.0,2604,IV GERES - Caruaru,De Risco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,260040 Água Preta,108,2,0,260040,Água Preta,260040.0,2603,III GERES - Palmares,De Risco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_partos_normal_munic.query('cd_munic == 261160')

,Munic Residencia PE,2601 Recife,2602 Limoeiro,2603 Palmares,2604 Caruaru,2605 Garanhuns,2606 Arcoverde,2607 Salgueiro,2608 Petrolina,2609 Ouricuri,2610 Afogados da Ingazeira,2611 Serra Talhada,2612 Goiana,cd_munic,municipio,codmunres,cod_geres,geresnome,TIPO_PARTO
132,261160 Recife,20854,24,20,15,6,3,4,4,1,0,3,4,261160,Recife,261160.0,2601,I GERES - Recife,Normal


#### 2. Dataset de Partos por Município de residência por estabelecimento (hospital)

In [6]:
df_estab_raiz = pd.read_csv(f"{DATADIR}/estabelecimentos.csv", sep=";", usecols=['Estabelecimento', 'Município', 'lat', 'lng'])
df_estab = df_estab_raiz.merge(df_partos_munic[['cd_munic', 'municipio']].drop_duplicates(), how='left', left_on='Município', right_on='municipio')
df_estab.head(3)

,Estabelecimento,Município,lat,lng,cd_munic,municipio
0,HOSPITAL DAS CLINICAS,Recife,-8.04742,-34.94850,261160,Recife
1,HOSPITAL AGAMENON MAGALHAES,Recife,-8.03029,-34.90931,261160,Recife
2,IMIP,Recife,-8.04824,-34.89970,261160,Recife


In [7]:
def transform_estab_df(df):
    df.drop('Total', axis=1, inplace=True)
    df['cd_estab'] = df['Hospital PE (CNES)'].apply(lambda x: x[:7])
    df['estabelecimento'] = df['Hospital PE (CNES)'].apply(lambda x: x[8:])
    return df

In [8]:
df_partos_risco_estab = pd.read_csv(f"{DATADIR}/partos_gravidez_risco_por_estabelecimento.csv", sep=";")
transform_estab_df(df_partos_risco_estab)
df_partos_risco_estab['TIPO_PARTO'] = 'Risco'

df_partos_norma_estab = pd.read_csv(f"{DATADIR}/partos_normais_por_estabelecimento.csv", sep=";")
transform_estab_df(df_partos_norma_estab)
df_partos_norma_estab['TIPO_PARTO'] = 'Normal'

df_partos_estab = pd.concat([df_partos_risco_estab, df_partos_norma_estab])
df_partos_estab.head()

,Hospital PE (CNES),2019,2020,2021,2022,cd_estab,estabelecimento,TIPO_PARTO
0,0000396 HOSPITAL DAS CLINICAS,785,811,1026,883,0000396,HOSPITAL DAS CLINICAS,Risco
1,0000418 HOSPITAL AGAMENON MAGALHAES,1734,1637,1782,1683,0000418,HOSPITAL AGAMENON MAGALHAES,Risco
2,0000434 IMIP,2718,980,949,2397,0000434,IMIP,Risco
3,2351994 HOSPITAL DE CARUARU JESUS NAZARENO,443,521,1240,1698,2351994,HOSPITAL DE CARUARU JESUS NAZARENO,Risco
4,2427427 HOSPITAL BARAO DE LUCENA,2167,2187,2389,2295,2427427,HOSPITAL BARAO DE LUCENA,Risco


#### 3. Export

In [9]:
df_partos_munic.to_parquet(f'{DATADIR}/partos_por_municipio_2019_2022.parquet.gzip')
df_partos_estab.to_parquet(f'{DATADIR}/partos_por_estabelecimento.parquet.gzip')
